In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [3]:
from loguru import logger
from langchain.callbacks import FileCallbackHandler
from datetime import date

today = date.today()

logfile = f"logs/full_pipeline_{today.year}_{today.month}_{today.day}.log"
logger.add(logfile, format="{message}", colorize=False, enqueue=True)
handler = FileCallbackHandler(logfile)

In [ ]:
from lib import create_cube_selection_chain, fetch_cubes_descriptions, parse_all_cubes, fetch_cube_sample, fetch_dimensions_triplets, fetch_dimensions

cube_selection_settings = {
    "temperature": 0.5,
    "top_p": 0.5
}
cubes = fetch_cubes_descriptions()
dimensions = fetch_dimensions()
cube_selection_chain = create_cube_selection_chain(api_key=OPENAI_API_KEY, handler=handler, **cube_selection_settings)

# question = f"sum of emission of CO2 for industry between year 2009 and 2011"
question = f"what is the weather"

cube_selection_response = await cube_selection_chain.ainvoke({
    "cubes": cubes,
    "dimensions": dimensions,
    "question": question,
})
cube_selection_response = cube_selection_response['text']

logger.info("========== CUBES RESPONSE ================")
logger.info(f"{cube_selection_response}")

WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
2025-01-15 16:53:34.665 | INFO     | __main__:<module>:21 - ========== CUBES RESPONSE ================
2025-01-15 16:53:34.666 | INFO     | __main__:<module>:22 - The cube ID that best answers the question "what is the weather" is:

**None of the provided cubes directly address weather conditions.**

Justification:
The cubes listed primarily focus on environmental data related to forest conditions, pollution, greenhouse gas emissions, and other ecological metrics. While some cubes, such as "Forest fire danger" and "Air pollution along the A2 and A13," may incorporate meteorological data to inform their assessments, they do not provide direct information about current weather conditions. 

If you are looking for a specific cube related to weather data, none of the cubes listed would be suitable as they do not explicitly c

In [5]:
cubes

'@prefix schema: <http://schema.org/> .\n@prefix qudt: <http://qudt.org/schema/qudt/> .\n@prefix dct: <http://purl.org/dc/terms/> .\n@prefix sh: <http://www.w3.org/ns/shacl#> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix cube: <https://cube.link/> .\n\n<https://environment.ld.admin.ch/foen/gefahren-waldbrand-warnung/1> a cube:Cube ;\n   schema:name "Forest fire danger"@en ;\n   schema:description "The FOEN\'s forest fire danger warning contains the general daily forest fire danger based on meteorological data and the resulting forest fire indices."@en .\n<https://environment.ld.admin.ch/foen/gefahren-waldbrand-praeventionsmassnahmen-kantone/1> a cube:Cube ;\n   schema:name "Forest fire prevention measures"@en ;\n   schema:description "Contains the forest fire prevention measures currently in force in the cantons."@en .\n<https://environment.ld.admin.ch/foen/nfi/nfi_T-changes/cube/2024-1> a cube:Cube ;\n

In [6]:
selected_cubes = parse_all_cubes(cube_selection_response)
selected_cube = selected_cubes[0]

logger.info("========== SELECTED FIRST CUBE ================")
logger.info(selected_cube)

IndexError: list index out of range

In [13]:
cube_and_sample = fetch_cube_sample(selected_cube)

# print(cube_and_sample)

In [14]:
dimensions_triplets = fetch_dimensions_triplets(selected_cube)

# print(dimensions_triplets)

In [15]:
from lib import create_query_generation_chain

query_generation_settings = {
    "temperature": 0.2,
    "top_p": 0.1
}

generation_chain = create_query_generation_chain(api_key=OPENAI_API_KEY, handler=handler, **query_generation_settings)

query_generation_response = await generation_chain.ainvoke({
    "cube_and_sample": cube_and_sample,
    "dimensions_triplets": dimensions_triplets,
    "cube": selected_cube,
    "question": question,
})
query_generation_response = query_generation_response['text']

logger.info("========== QUERY GENERATION RESPONSE ================")
logger.info(f"{query_generation_response}")

WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
2025-01-15 16:52:25.760 | INFO     | __main__:<module>:18 - ========== QUERY GENERATION RESPONSE ================
2025-01-15 16:52:25.761 | INFO     | __main__:<module>:19 - PREFIX cube: <https://cube.link/>
PREFIX schema: <http://schema.org/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT (AVG(?emission) AS ?averageEmission)
WHERE {
    <https://environment.ld.admin.ch/foen/ubd000503/8> a cube:Cube;
        cube:observationSet ?observationSet.

    ?observationSet a cube:ObservationSet;
        cube:observation ?observation.

    ?observation a cube:Observation;
        <https://environment.ld.admin.ch/foen/ubd000503/j

In [16]:
from lib import run_query

result = run_query(query_generation_response)

logger.info("=========== QUERY RESULT ============")
logger.info(result)

2025-01-15 16:52:26.009 | INFO     | __main__:<module>:5 - =========== QUERY RESULT ============
2025-01-15 16:52:26.012 | INFO     | __main__:<module>:6 - {'head': {'vars': ['averageEmission']}, 'results': {'bindings': [{'averageEmission': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal', 'type': 'literal', 'value': '24.983333333333333333333333'}}]}}
